#### Use breast cancer dataset. The dataset contains a total number of 10 features labeled in either benign or malignant classes. The features have 699 instances out of which 16 feature values are missing. The dataset only contains numeric values. Implement the ensemble method using Decision tree classifiers.

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/rahul96rajan/sample_datasets/master/breast_cancer.csv')
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
X = data.drop('diagnosis', axis=1)
y = data['diagnosis']

In [4]:
X.shape, y.shape

((569, 32), (569,))

In [5]:
y.value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42)

In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398 entries, 149 to 102
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       398 non-null    int64  
 1   radius_mean              398 non-null    float64
 2   texture_mean             398 non-null    float64
 3   perimeter_mean           398 non-null    float64
 4   area_mean                398 non-null    float64
 5   smoothness_mean          398 non-null    float64
 6   compactness_mean         398 non-null    float64
 7   concavity_mean           398 non-null    float64
 8   concave points_mean      398 non-null    float64
 9   symmetry_mean            398 non-null    float64
 10  fractal_dimension_mean   398 non-null    float64
 11  radius_se                398 non-null    float64
 12  texture_se               398 non-null    float64
 13  perimeter_se             398 non-null    float64
 14  area_se                 

In [8]:
X_train.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
count,3.980000e+02,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,0.0
mean,3.012241e+07,14.176078,19.159171,92.256332,659.770854,0.095962,0.103303,0.088071,0.048440,0.180308,0.062664,0.404789,1.203627,2.875302,40.597329,0.007054,0.025356,0.032259,0.011926,0.020372,0.003791,16.287322,25.505553,107.363844,881.848241,0.131772,0.249086,0.268497,0.113839,0.288300,0.083537,NaN
std,1.270358e+08,3.535730,4.214074,24.289529,358.865452,0.013766,0.051209,0.077836,0.037712,0.026871,0.007248,0.290452,0.519348,2.114108,48.883639,0.003114,0.017995,0.030055,0.006234,0.008016,0.002651,4.799691,6.017564,33.083204,570.753903,0.023023,0.148622,0.196139,0.063886,0.062318,0.017518,NaN
min,8.915000e+03,7.691000,9.710000,48.340000,170.400000,0.062510,0.019380,0.000000,0.000000,0.116700,0.049960,0.111500,0.360200,0.757000,6.802000,0.002667,0.002252,0.000000,0.000000,0.007882,0.000895,8.678000,12.020000,54.490000,223.600000,0.081250,0.034320,0.000000,0.000000,0.156500,0.055040,NaN
25%,8.682072e+05,11.752500,16.170000,75.805000,426.175000,0.085550,0.063407,0.029585,0.020692,0.161575,0.057665,0.236150,0.836950,1.679250,18.160000,0.005080,0.012782,0.015180,0.007910,0.015200,0.002251,13.075000,21.070000,84.542500,521.550000,0.114325,0.147325,0.116475,0.063885,0.247775,0.071155,NaN
50%,9.053455e+05,13.275000,18.700000,85.980000,546.250000,0.094625,0.091280,0.061880,0.034110,0.178800,0.061325,0.319400,1.108000,2.278500,24.830000,0.006413,0.020500,0.026140,0.011160,0.018695,0.003219,14.975000,25.155000,97.745000,687.600000,0.131150,0.209250,0.226200,0.099270,0.280750,0.080165,NaN
75%,8.813066e+06,16.090000,21.555000,105.625000,797.400000,0.104475,0.130575,0.127075,0.073963,0.195550,0.066007,0.469125,1.439750,3.277750,44.410000,0.008200,0.031790,0.042500,0.014605,0.022812,0.004509,18.707500,29.410000,125.300000,1061.250000,0.146000,0.341600,0.384700,0.160400,0.315650,0.091745,NaN
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.311400,0.426800,0.201200,0.304000,0.097440,2.873000,3.647000,21.980000,542.200000,0.031130,0.135400,0.396000,0.052790,0.061460,0.029840,36.040000,49.540000,251.200000,4254.000000,0.218400,0.937900,0.903400,0.275600,0.663800,0.173000,NaN


In [9]:
pca = PCA(n_components=5, random_state=42)

def preprop(df):
    xdata = df.copy()
    xdata.drop(['id','Unnamed: 32'], axis=1, inplace=True)
    xdata = pca.fit_transform(xdata)
    return xdata

In [10]:
X_train = preprop(X_train)

In [11]:
dtc = DecisionTreeClassifier(random_state=42, max_depth=2)
bag_clf = BaggingClassifier(bootstrap=True, n_jobs=-1, oob_score=True)

gs_params = {'n_estimators':[400, 800], 'max_samples':[50, 100]}

gs = GridSearchCV(bag_clf, gs_params, scoring='f1_weighted', n_jobs=-1, cv=4)

In [12]:
gs.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=BaggingClassifier(n_jobs=-1, oob_score=True),
             n_jobs=-1,
             param_grid={'max_samples': [50, 100], 'n_estimators': [400, 800]},
             scoring='f1_weighted')

In [13]:
print(gs.best_params_)
print(gs.best_score_)
print('OOB score:', gs.best_estimator_.oob_score_)

{'max_samples': 100, 'n_estimators': 400}
0.9418108870454094
OOB score: 0.9396984924623115


In [14]:
y_pred_train = gs.predict(X_train)
X_test = preprop(X_test)
y_pred_test = gs.predict(X_test)

In [15]:
print('F1 Score(Train): {:.4f}'.format(f1_score(y_train, y_pred_train,
                                                average='weighted')))
print('F1 Score(Test): {:.4f}'.format(f1_score(y_test, y_pred_test,
                                               average='weighted')))

print('\nAccuracy Score(Train): {:.4f}'.format(accuracy_score(y_train,
                                                              y_pred_train)))
print('Accuracy Score(Test): {:.4f}'.format(accuracy_score(y_test,
                                                           y_pred_test)))

F1 Score(Train): 0.9571
F1 Score(Test): 0.8840

Accuracy Score(Train): 0.9573
Accuracy Score(Test): 0.8830
